In [1]:
import requests
import xmltodict
import pandas as pd
import numpy as np
import math
from collections import Counter
from statistics import mean


In [2]:
df_ = pd.DataFrame()
urllist = ["http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=" + str(i) for i in range(1,101)]


['http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=1',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=2',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=3',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=4',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=5',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=6',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=7',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=8',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=9',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=10',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=11',
 'http://www.bestbuy.ca/api/v2/XML/search?categoryid=20352&pagesize=100&page=12',
 'http://www.bestbuy.ca/a

In [3]:
count=0 

for url in urllist: 
    response_new = requests.get(url)
    
    if count == 0: 
        df = pd.DataFrame(xmltodict.parse(response_new.text)['SearchResponse']['Products']['Product'])
        df['Page'] = 1
        count += 1 
    
    else: 
        df2 = pd.DataFrame(xmltodict.parse(response_new.text)['SearchResponse']['Products']['Product'])
        df2['Page'] = count + 1 
        df = df.append(df2)
        count += 1 

df = df.reset_index()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
cv = CountVectorizer(stop_words='english')
catname = df['CategoryName'].unique()
catname

array(['Laptops', 'Chromebooks', 'Gaming Laptops', 'Apple MacBook Air',
       '2 in 1 Laptops', 'Apple MacBook', 'Apple MacBook Pro'],
      dtype=object)

In [5]:
keyword = {} 

for i in range (0, len(catname)):
    dat = df.loc[df['CategoryName']==catname[i]]['SeoText']
    wordcount = cv.fit_transform(dat).toarray()
    wordcount = pd.DataFrame(wordcount, columns=cv.get_feature_names())
    wordlist = wordcount.sum(axis=0).sort_values(ascending=False).index.tolist()
    
    keyword[catname[i]] = wordlist

print(keyword.keys())
print(keyword[catname[2]])


dict_keys(['Laptops', 'Chromebooks', 'Gaming Laptops', 'Apple MacBook Air', '2 in 1 Laptops', 'Apple MacBook', 'Apple MacBook Pro'])
['gaming', 'ram', 'laptop', 'custom', 'intel', 'ssd', 'i7', 'nvidia', '9750h', 'rtx', 'pcie', '1tb', 'core', 'entertainment', 'hp', 'asus', '16gb', '2tb', 'omen', '32gb', 'hdd', 'sata', '64gb', 'gtx', '10', 'win', '512gb', '3xusb', '15', '2070', 'rog', 'ti', '1660', 'msi', '8gb', 'amd', 'ryzen', '8750h', '3750h', '15t', 'max', 'strix', '2080', 'i9', '2060', 'pro', 'home', '9880h', 'webcam', 'gigabyte', 'tuf', '17t', 'aorus', '256gb', 'dc100', 'iii', 'scar', 'g531gw', 'raid', '1650', 'gl731gt', '505du', '2s', 'zephyrus', '4tb', 'stealth', '2xusb', 'refurbished', 'i5', 'uhd', '17', 'gs75', '205', '620', 'raider', 'fx705dt', 'gx502gv', 'gl75', '9sdk', '057', 'certified', 'classic', 'aero', 'dell', 'tuf705du', 'kh74', 'lenovo', 'g6', '450', '8265u', 'probook', 'open', 'box', 'gp75', 'business', '9sd', '437', 'thinkpad', '2x1tb', 'geforce', 'black', 'fhd', 'wi

## Seller Extraction

In [6]:
notBB = df[~df['Seller'].isnull()]
BB = df[df['Seller'].isnull()]

seller = [notBB['Seller'].iloc[i].get('Name') for i in range(0,len(notBB))]
sellunique =[] 
numsell = {} 

for x in seller: 
    if x not in sellunique: 
        sellunique.append(x)
        numsell[x] = 1
    if x in sellunique: 
        numsell[x] += 1 

sellunique
numsell

{'Microcad Computer Corporation': 60,
 'TVOUTLET': 33,
 'Best Buy Business': 263,
 'SaveOnMacs': 36,
 'Olde Time Mac': 36,
 'NetTradez': 503,
 'ORION COMPUTER AND ELECTRONICS': 12,
 'PRO Open Box': 215,
 'THE PC ROOM': 98,
 'Electronics_Canada': 18,
 'DCR': 163,
 'Certified Refurbisher': 34,
 'CDI Computer Dealers Inc.': 334,
 'ALC Micro': 34,
 'OEM': 37,
 'Wireless Experts, Inc.': 2,
 'Canada Wholesale': 10,
 'Aero Malls Inc.': 26,
 'IGO Electronics': 85,
 'DirectEASYBUY': 83,
 'Mike’s Computer Shop': 345,
 'AlwaysTech': 14,
 'TSC Deals': 18,
 'Mega PC Mall': 2,
 'OpenBox': 13,
 'CPR upper beeches toronto': 2,
 'RefurbIT': 40,
 'MCI ONLINE': 37,
 'MFG DIRECT ELECTRONICS': 9,
 'TecnoCanada': 16,
 'Canadian Outlet': 14,
 'Tecnodeals': 108,
 'Computer Depot': 41,
 'QR Computers': 18,
 'Advanced Skyline Technology Ltd.': 70,
 'ACM Électronique': 17,
 'Techdealz': 8,
 'SIPL Online': 30,
 'OneDealOutlet Canada': 182,
 'Lenovo Canada': 66,
 'Exooto Media': 13,
 'Advanced Computers': 139,
 'S

## Description Score

In [10]:
desclen = [len(notBB['ShortDescription'].iloc[i].split()) for i in range(0,len(notBB))]
uniqlen = [len(set(notBB['ShortDescription'].iloc[i].split())) for i in range(0,len(notBB))]

BBlen = [len((BB['ShortDescription'].iloc[i].split())) for i in range(0,len(BB))]
BBuniqlen = [len(set(BB['ShortDescription'].iloc[i].split())) for i in range(0,len(BB))]
print(mean(uniqlen))
print(mean(BBuniqlen))

48.16562665256478
51.14862385321101


In [11]:
numworddiff = [i/mean(BBlen) for i in desclen]

In [12]:
shortdesc = notBB['ShortDescription']
wordcount = cv.fit_transform(shortdesc).toarray()
wordcount = pd.DataFrame(wordcount, columns=cv.get_feature_names())
wordlist = wordcount.sum(axis=0).sort_values(ascending=False).index.tolist()
wordcount.sum(axis=0).sort_values(ascending=False)[0:5]

hd       9448
10       8097
intel    7057
ram      6911
ssd      6668
dtype: int64

In [13]:
BBshortdesc = BB['ShortDescription']
BBwordcount = cv.fit_transform(BBshortdesc).toarray()
BBwordcount = pd.DataFrame(BBwordcount, columns=cv.get_feature_names())
BBwordlist = BBwordcount.sum(axis=0).sort_values(ascending=False).index.tolist()
BBwordcount.sum(axis=0).sort_values(ascending=False)[0:5]

laptop       586
strong       470
processor    426
core         389
ram          383
dtype: int64

In [14]:
wordinlist = [[j for j in notBB['ShortDescription'].iloc[i].split() 
               if j in BBwordlist] 
              for i in range(0,len(notBB))]

BBinlist = [len([j for j in BB['ShortDescription'].iloc[i].split() 
                 if j in BBwordlist]) 
            for i in range(0,len(BB))]

propinlist = [len(wordinlist[i])/uniqlen[i]
              for i in range(0,len(wordinlist))]

In [15]:
notBB['sellername'] = seller
#notBB['length'] = desclen
#notBB['uniquelength'] = uniqlen
notBB['proportionlength'] = numworddiff
#notBB['wordsinlist'] = wordinlist
notBB['proportioninlist'] = propinlist
notBB['descrscore'] = (notBB['proportionlength'] + notBB['proportioninlist'])/2

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

## Title Score

In [16]:
titlen = [len((notBB['Name'].iloc[i].split())) for i in range(0,len(notBB))]
BBtitlen = [len((BB['Name'].iloc[i].split())) for i in range(0,len(BB))]

tituniqlen = [len(set(notBB['Name'].iloc[i].split())) for i in range(0,len(notBB))]
BBtituniqlen = [len(set(BB['Name'].iloc[i].split())) for i in range(0,len(BB))]
titlediff = [max(0,min(min(2-i/mean(BBtitlen),1),i/mean(BBtitlen))) for i in titlen]

print(mean(titlen))
print(mean(BBtitlen))

18.896245372818615
14.906422018348623


In [17]:
title = notBB['Name']
titlecount = cv.fit_transform(title).toarray()
titlecount = pd.DataFrame(titlecount, columns=cv.get_feature_names())
titlelist = titlecount.sum(axis=0).sort_values(ascending=False).index.tolist()
titlecount.sum(axis=0).sort_values(ascending=False)[0:5]

ram       7487
laptop    6401
intel     6299
ssd       6279
custom    5061
dtype: int64

In [18]:
BBtitle = BB['Name']
BBtitlecount = cv.fit_transform(BBtitle).toarray()
BBtitlecount = pd.DataFrame(BBtitlecount, columns=cv.get_feature_names())
BBtitlelist = BBtitlecount.sum(axis=0).sort_values(ascending=False).index.tolist()
BBtitlecount.sum(axis=0).sort_values(ascending=False)[0:5]

ram       541
intel     450
laptop    438
ssd       402
core      399
dtype: int64

In [19]:
titleinlist = [[j for j in notBB['Name'].iloc[i].split() 
               if j.lower() in keyword[notBB['CategoryName'].iloc[i]]] 
               for i in range(0,len(notBB))]

BBtitleinlist = [len([j for j in BB['Name'].iloc[i].split() 
                      if j.lower() in keyword[notBB['CategoryName'].iloc[i]]]) 
                 for i in range(0,len(BB))]

proptitleinlist = [len(titleinlist[i])/tituniqlen[i] 
                   for i in range(0,len(wordinlist))]

#proptitleinlist = [len(titleinlist[i])/tituniqlen[i] * (mean(BBtitleinlist)/mean(BBtituniqlen)) for i in range(0,len(wordinlist))]

In [20]:
#notBB['titlelength'] = titlen
notBB['proportiontitle'] = titlediff
#notBB['titleinlist'] = titleinlist
notBB['proptitleinlist'] = proptitleinlist
notBB['titlescore'] = (notBB['proportiontitle'] + notBB['proptitleinlist'])/2

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [21]:
notBB.head()

,index,CategoryName,CustomerRating,CustomerRatingCount,CustomerReviewCount,DisplayEndDateTime,DisplayStartDateTime,Ehf,HasFrenchContent,HasPromotion,...,ThumbnailImage,highResImage,isVisible,sellername,proportionlength,proportioninlist,descrscore,proportiontitle,proptitleinlist,titlescore
18,18,Laptops,4,3,3,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,https://multimedia.bbycastatic.ca/multimedia/p...,https://multimedia.bbycastatic.ca/multimedia/p...,true,Microcad Computer Corporation,0.209119,0.000000,0.104560,0.524126,0.833333,0.678730
26,26,Chromebooks,0,0,0,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,https://multimedia.bbycastatic.ca/multimedia/p...,https://multimedia.bbycastatic.ca/multimedia/p...,true,Microcad Computer Corporation,0.400812,0.086957,0.243884,0.725382,0.526316,0.625849
29,29,Chromebooks,0,0,0,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,https://multimedia.bbycastatic.ca/multimedia/p...,https://multimedia.bbycastatic.ca/multimedia/p...,true,Microcad Computer Corporation,0.505372,0.000000,0.252686,0.658296,0.550000,0.604148
35,35,Apple MacBook Pro,0,0,0,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,https://multimedia.bbycastatic.ca/multimedia/p...,https://multimedia.bbycastatic.ca/multimedia/p...,true,TVOUTLET,0.975891,0.204082,0.589986,0.524126,0.500000,0.512063
42,42,Laptops,2,1,1,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,https://multimedia.bbycastatic.ca/multimedia/p...,https://multimedia.bbycastatic.ca/multimedia/p...,true,Microcad Computer Corporation,0.714491,0.184211,0.449351,0.524126,0.681818,0.602972


## Image

In [22]:
notBB['imgscore'] = 0 

for i in range(0,notBB.shape[0]):
    if notBB.highResImage.iloc[i] != np.nan: 
        notBB.imgscore.iloc[i] = 1 
    elif notBB.ThumbnailImage.iloc[i] != np.nan: 
        notBB.imgscore.iloc[i] = 0.5 
    else:
        notBB.imgscore.iloc[i] = 0 

notBB['imgscore'].head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guid

18    1
26    1
29    1
35    1
42    1
Name: imgscore, dtype: int64

## What's in the Box & Warranty

In [23]:
product = pd.read_csv('product_wib.csv')
product2 = product[['Name','HasWarranty','WhatsInTheBox']]

In [24]:
merged = notBB.merge(product2, on='Name')
merged.shape

(376, 45)

In [25]:
merged['warrantyscore'] = 0 

for i in range(0,merged.shape[0]):
    if merged.HasWarranty.iloc[i] ==True: 
        merged.warrantyscore.iloc[i] = 1 

merged['warrantyscore']

0      0
1      0
2      0
3      0
4      0
      ..
371    0
372    0
373    0
374    0
375    0
Name: warrantyscore, Length: 376, dtype: int64

In [26]:
merged['inboxscore'] = 0 

for i in range(0,merged.shape[0]):
    if merged.WhatsInTheBox.iloc[i] !=np.nan: 
        merged.inboxscore.iloc[i] = 1 

merged['inboxscore']

0      1
1      1
2      1
3      1
4      1
      ..
371    1
372    1
373    1
374    1
375    1
Name: inboxscore, Length: 376, dtype: int64

## Product Spec

In [27]:
#product = pd.read_csv('product_spec.csv')
#product.columns
#len(product['Specs'][0])

Due to time-constraint, we consider generating synthetic data in calculating product specification score.

In [28]:
merged['specscore'] = np.random.uniform(0,1,merged.shape[0])
merged['specscore']

0      0.831598
1      0.374945
2      0.125991
3      0.129756
4      0.522988
         ...   
371    0.286910
372    0.806433
373    0.202911
374    0.222560
375    0.318376
Name: specscore, Length: 376, dtype: float64

## Final Score

In [29]:
notBB.columns

Index(['index', 'CategoryName', 'CustomerRating', 'CustomerRatingCount',
       'CustomerReviewCount', 'DisplayEndDateTime', 'DisplayStartDateTime',
       'Ehf', 'HasFrenchContent', 'HasPromotion', 'HideSaleEndDate',
       'HideSavings', 'IsAdvertised', 'IsClearance', 'IsFrenchCompliant',
       'IsInStoreOnly', 'IsMarketplace', 'IsOnlineOnly', 'IsPreorderable',
       'IsPriceEndsLabel', 'Name', 'Page', 'PriceUnit', 'ProductUrl',
       'RegularPrice', 'RequiresAgeVerification', 'SaleEndDateTime',
       'SalePrice', 'Seller', 'SeoText', 'ShortDescription', 'Sku',
       'ThumbnailImage', 'highResImage', 'isVisible', 'sellername',
       'proportionlength', 'proportioninlist', 'descrscore', 'proportiontitle',
       'proptitleinlist', 'titlescore', 'imgscore'],
      dtype='object')

In [30]:
## Large Data Set Score
description_weight =  15
title_weight = 20
image_weight = 20

allweight = [description_weight,title_weight,image_weight] 
notBB['final_score'] = (description_weight*notBB['descrscore']+ title_weight*notBB['titlescore'] + image_weight*notBB['imgscore'])/sum(allweight)


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


## Subset Data Score

In [31]:
merged.columns

Index(['index', 'CategoryName', 'CustomerRating', 'CustomerRatingCount',
       'CustomerReviewCount', 'DisplayEndDateTime', 'DisplayStartDateTime',
       'Ehf', 'HasFrenchContent', 'HasPromotion', 'HideSaleEndDate',
       'HideSavings', 'IsAdvertised', 'IsClearance', 'IsFrenchCompliant',
       'IsInStoreOnly', 'IsMarketplace', 'IsOnlineOnly', 'IsPreorderable',
       'IsPriceEndsLabel', 'Name', 'Page', 'PriceUnit', 'ProductUrl',
       'RegularPrice', 'RequiresAgeVerification', 'SaleEndDateTime',
       'SalePrice', 'Seller', 'SeoText', 'ShortDescription', 'Sku',
       'ThumbnailImage', 'highResImage', 'isVisible', 'sellername',
       'proportionlength', 'proportioninlist', 'descrscore', 'proportiontitle',
       'proptitleinlist', 'titlescore', 'imgscore', 'HasWarranty',
       'WhatsInTheBox', 'warrantyscore', 'inboxscore', 'specscore'],
      dtype='object')

In [32]:
## Large Data Set Score
description_weight =  20
title_weight = 25
image_weight = 25
spec_weight = 20
warranty_weight = 5 
inbox_weight = 5 

allweight = [description_weight,title_weight,image_weight,warranty_weight,inbox_weight,spec_weight] 
merged['final_score'] = (description_weight*merged['descrscore']+ title_weight*merged['titlescore'] + image_weight*merged['imgscore'] + warranty_weight*merged['warrantyscore'] + inbox_weight*merged['inboxscore'] + spec_weight*merged['specscore'] )/sum(allweight)
merged.head()

,index,CategoryName,CustomerRating,CustomerRatingCount,CustomerReviewCount,DisplayEndDateTime,DisplayStartDateTime,Ehf,HasFrenchContent,HasPromotion,...,proportiontitle,proptitleinlist,titlescore,imgscore,HasWarranty,WhatsInTheBox,warrantyscore,inboxscore,specscore,final_score
0,18,Laptops,4,3,3,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,0.524126,0.833333,0.678730,1,False,"HP 14"" EliteBook 840 G3 Laptop",0,1,0.831598,0.656914
1,26,Chromebooks,0,0,0,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,0.725382,0.526316,0.625849,1,False,"Chromebook, Power Adapter",0,1,0.374945,0.580228
2,29,Chromebooks,0,0,0,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,0.658296,0.550000,0.604148,1,False,"Chromebook, Power adapter, Wireless Mouse, Lap...",0,1,0.125991,0.526773
3,42,Laptops,2,1,1,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,0.524126,0.681818,0.602972,1,False,"Laptop, Power Cable",0,1,0.129756,0.566564
4,53,2 in 1 Laptops,5,1,1,{'@xsi:nil': 'true'},{'@xsi:nil': 'true'},0,false,true,...,0.859552,0.470588,0.665070,1,False,"['HP 13.3"" EliteBook x360 1030 G2', 'Power Ada...",0,1,0.522988,0.683052


In [33]:
notBB['final_score'].describe()

count    9455.000000
mean        0.736253
std         0.057743
min         0.407042
25%         0.699743
50%         0.745151
75%         0.777625
max         0.953361
Name: final_score, dtype: float64

In [34]:
len(merged)

376

## Converting Score Data

In [35]:
merged.columns

Index(['index', 'CategoryName', 'CustomerRating', 'CustomerRatingCount',
       'CustomerReviewCount', 'DisplayEndDateTime', 'DisplayStartDateTime',
       'Ehf', 'HasFrenchContent', 'HasPromotion', 'HideSaleEndDate',
       'HideSavings', 'IsAdvertised', 'IsClearance', 'IsFrenchCompliant',
       'IsInStoreOnly', 'IsMarketplace', 'IsOnlineOnly', 'IsPreorderable',
       'IsPriceEndsLabel', 'Name', 'Page', 'PriceUnit', 'ProductUrl',
       'RegularPrice', 'RequiresAgeVerification', 'SaleEndDateTime',
       'SalePrice', 'Seller', 'SeoText', 'ShortDescription', 'Sku',
       'ThumbnailImage', 'highResImage', 'isVisible', 'sellername',
       'proportionlength', 'proportioninlist', 'descrscore', 'proportiontitle',
       'proptitleinlist', 'titlescore', 'imgscore', 'HasWarranty',
       'WhatsInTheBox', 'warrantyscore', 'inboxscore', 'specscore',
       'final_score'],
      dtype='object')

In [36]:
merged2 = merged[['CategoryName','sellername', 'Name', 'proportionlength', 'proportioninlist','descrscore','proportiontitle',
       'proptitleinlist','titlescore','imgscore','warrantyscore', 'inboxscore', 'final_score',
       'specscore','Page']]

merged2=merged2.rename(columns = {'CategoryName':'subcategory','sellername':'seller','Name':'name','Page':'page','final_score':'overallscore'})
merged2['category'] = 'Laptops'
merged2['start'] = 0 
merged2['end'] = 1
merged2.columns

Index(['subcategory', 'seller', 'name', 'proportionlength', 'proportioninlist',
       'descrscore', 'proportiontitle', 'proptitleinlist', 'titlescore',
       'imgscore', 'warrantyscore', 'inboxscore', 'overallscore', 'specscore',
       'page', 'category', 'start', 'end'],
      dtype='object')

In [38]:
varlist = ['overall','descr','title','img','warranty','inbox','spec']
scorelist = [i + 'score' for i in varlist]
score25 = [i + 'score25th' for i in varlist]
score50 = [i + 'score50th' for i in varlist]
score75 = [i + 'score75th' for i in varlist]
scorestate = [i + '_state' for i in varlist]

In [39]:
for i in range (0,len(varlist)): 
    merged2[score25[i]] = merged2[scorelist[i]].describe()[4]
    merged2[score50[i]] = merged2[scorelist[i]].describe()[5]
    merged2[score75[i]] = merged2[scorelist[i]].describe()[6]
    merged2[scorestate[i]] = 'Awesome'


In [40]:
merged2['titlescore']

0      0.678730
1      0.625849
2      0.604148
3      0.602972
4      0.665070
         ...   
371    0.570606
372    0.870952
373    0.753305
374    0.757344
375    0.713319
Name: titlescore, Length: 376, dtype: float64

In [42]:
for i in range(0,merged.shape[0]):
    if merged2.overallscore.iloc[i] < merged2.overallscore25th.iloc[i]: 
        merged2.overall_state.iloc[i] = 'Need Improvement'
    elif merged2.overallscore.iloc[i] < merged2.overallscore50th.iloc[i]: 
        merged2.overall_state.iloc[i] = 'Scope for Improvement'
    elif merged2.overallscore.iloc[i] < merged2.overallscore75th.iloc[i]: 
        merged2.overall_state.iloc[i] = 'Almost There'

In [43]:
for i in range(0,merged.shape[0]):
    if merged2.descrscore.iloc[i] < merged2.descrscore25th.iloc[i]: 
        merged2.descr_state.iloc[i] = 'Need Improvement'
    elif merged2.descrscore.iloc[i] < merged2.descrscore50th.iloc[i]: 
        merged2.descr_state.iloc[i] = 'Scope for Improvement'
    elif merged2.descrscore.iloc[i] < merged2.descrscore75th.iloc[i]: 
        merged2.descr_state.iloc[i] = 'Almost There'

In [44]:
for i in range(0,merged.shape[0]):
    if merged2.titlescore.iloc[i] < merged2.titlescore25th.iloc[i]: 
        merged2.title_state.iloc[i] = 'Need Improvement'
    elif merged2.titlescore.iloc[i] < merged2.titlescore50th.iloc[i]: 
        merged2.title_state.iloc[i] = 'Scope for Improvement'
    elif merged2.titlescore.iloc[i] < merged2.titlescore75th.iloc[i]: 
        merged2.title_state.iloc[i] = 'Almost There'
        

In [45]:
for i in range(0,merged.shape[0]):
    if merged2.imgscore.iloc[i] < merged2.imgscore25th.iloc[i]: 
        merged2.img_state.iloc[i] = 'Need Improvement'
    elif merged2.imgscore.iloc[i] < merged2.imgscore50th.iloc[i]: 
        merged2.img_state.iloc[i] = 'Scope for Improvement'
    elif merged2.imgscore.iloc[i] < merged2.imgscore75th.iloc[i]: 
        merged2.img_state.iloc[i] = 'Almost There'

In [46]:
for i in range(0,merged.shape[0]):
    if merged2.warrantyscore.iloc[i] <= merged2.warrantyscore25th.iloc[i]: 
        merged2.warranty_state.iloc[i] = 'Need Improvement'
    elif merged2.warrantyscore.iloc[i] <= merged2.warrantyscore50th.iloc[i]: 
        merged2.warranty_state.iloc[i] = 'Scope for Improvement'
    elif merged2.warrantyscore.iloc[i] <= merged2.warrantyscore75th.iloc[i]: 
        merged2.warranty_state.iloc[i] = 'Almost There'

In [47]:
for i in range(0,merged.shape[0]):
    if merged2.inboxscore.iloc[i] < merged2.inboxscore75th.iloc[i]: 
        merged2.inbox_state.iloc[i] = 'Almost There'
    elif merged2.inboxscore.iloc[i] < merged2.inboxscore50th.iloc[i]: 
        merged2.inbox_state.iloc[i] = 'Scope for Improvement'
    elif merged2.inboxscore.iloc[i] < merged2.inboxscore25th.iloc[i]: 
        merged2.inbox_state.iloc[i] = 'Need Improvement'

In [48]:
for i in range(0,merged.shape[0]):
    if merged2.specscore.iloc[i] < merged2.specscore75th.iloc[i]: 
        merged2.spec_state.iloc[i] = 'Almost There'
    elif merged2.specscore.iloc[i] < merged2.specscore50th.iloc[i]: 
        merged2.spec_state.iloc[i] = 'Scope for Improvement'
    elif merged2.specscore.iloc[i] < merged2.specscore25th.iloc[i]: 
        merged2.spec_state.iloc[i] = 'Need Improvement' 

In [49]:
merged2.to_csv('score.csv')

## Product & Score

In [62]:
notBB[notBB['sellername']=='Vancouver Electronic Depot']['ShortDescription'].iloc[0]

'Refurbished. MacBook Pro 13" Retina 2.9GHz i5 16GB / 256GB - 2015 Model - Grade A, Excellent Condition, 9/10!'

In [67]:
merged2[merged2['seller']=='Vancouver Electronic Depot']

,subcategory,seller,name,proportionlength,proportioninlist,descrscore,proportiontitle,proptitleinlist,titlescore,imgscore,...,warrantyscore75th,warranty_state,inboxscore25th,inboxscore50th,inboxscore75th,inbox_state,specscore25th,specscore50th,specscore75th,spec_state
311,Apple MacBook Pro,Vancouver Electronic Depot,"MacBook Pro 13"" Retina 2.9GHz i5 16GB / 256GB ...",0.331106,0.055556,0.193331,0.725382,0.555556,0.640469,1,...,0.0,Need Improvement,1.0,1.0,1.0,Awesome,0.258346,0.483624,0.75737,Almost There


In [63]:
merged3 = merged2.groupby(['seller']).mean()
merged3['overallscore'].sort_values()

seller
Vancouver Electronic Depot          0.549535
Etech                               0.561097
Advanced Skyline Technology Ltd.    0.567995
OEM                                 0.572126
Electronics_Canada                  0.577410
SIPL Online                         0.583869
BigOnDeals                          0.585006
UNIWAY                              0.595540
SaveOnMacs                          0.600980
TVOUTLET                            0.602495
Olde Time Mac                       0.603218
NetTradez                           0.612386
Deal Targets                        0.613825
Laptopking                          0.616542
RefurbIT                            0.617548
Microcad Computer Corporation       0.619783
COMPUPOINT INC.                     0.621653
TSC Deals                           0.624820
X-Dynamic Systems                   0.626869
Square PC                           0.629996
CDI Computer Dealers Inc.           0.634694
IGO Electronics                     0.634765
ORI